In [1]:
import requests
import json
import lxml.html as lh
import pandas as pd

from bs4 import BeautifulSoup as bs

from investool.ticker import isin2ft

In [2]:
ft_url = 'http://markets.ft.com'

In [3]:
# Get Fund historical prices from markets.ft.com

def get_historical_prices( isin_ticker, fromDate='2018/01/01', toDate='2018/11/27' ):   
    
    q_params = {'startDate': fromDate,
                'endDate': toDate,
                'symbol': isin2ft(isin_ticker)}

    # Download the result from website
    r = requests.get( ft_url + '/data/equities/ajax/get-historical-prices', params = q_params)
    #print(r.url)

    # Check if it is ok
    if r.status_code == requests.codes.ok:

        # Prepare table and headers
        headers = ["Date", "Open", "High", "Low", "Close", "Volume"]
        col = []
        for h in headers:
            col.append((h,[]))

        #Store the contents of the website under doc
        doc = lh.fromstring(r.json()['html'])
    
        #Parse data that are stored between <tr>..</tr> of HTML
        tr_elements = doc.xpath('//tr')
    
        #Since out first row is the header, data is stored on the second row onwards
        for e in tr_elements:
            #If row is not of size 6, the //tr data is not from our table 
            if len(e)!=6:
                break
    
            #i is the index of our column
            i=0
    
            #Iterate through each element of the row
            for t in e.iterchildren():
         
                if i==0:
                    data=t[1].text_content()
                else:
                    data=t.text_content() 
                
                #Append the data to the empty list of the i'th column
                col[i][1].append(data)
        
                #Increment i for the next column
                i+=1

        return {title:column for (title,column) in col}   


In [12]:
df = pd.DataFrame(get_historical_prices('LU0270844359',fromDate='2015/01/01',toDate='2018/12/31'))

df['Date'] = pd.to_datetime(df['Date'])
df['Open'] = pd.to_numeric(df['Open'].replace({',':''},regex=True))
df['High'] = pd.to_numeric(df['High'].replace({',':''},regex=True))
df['Low'] = pd.to_numeric(df['Low'].replace({',':''},regex=True))
df['Close'] = pd.to_numeric(df['Close'].replace({',':''},regex=True))


In [13]:
df.plot(x='Date',y='Close')

In [14]:
df

Date  Open  High   Low  Close Volume
0    2018-11-30  9.38  9.38  9.38   9.38   ----
1    2018-11-29  9.37  9.37  9.37   9.37   ----
2    2018-11-28  9.38  9.38  9.38   9.38   ----
3    2018-11-27  9.38  9.38  9.38   9.38   ----
4    2018-11-26  9.38  9.38  9.38   9.38   ----
5    2018-11-23  9.34  9.34  9.34   9.34   ----
6    2018-11-22  9.37  9.37  9.37   9.37   ----
7    2018-11-21  9.37  9.37  9.37   9.37   ----
8    2018-11-20  9.34  9.34  9.34   9.34   ----
9    2018-11-19  9.38  9.38  9.38   9.38   ----
10   2018-11-16  9.35  9.35  9.35   9.35   ----
11   2018-11-15  9.38  9.38  9.38   9.38   ----
12   2018-11-14  9.28  9.28  9.28   9.28   ----
13   2018-11-13  9.31  9.31  9.31   9.31   ----
14   2018-11-12  9.24  9.24  9.24   9.24   ----
15   2018-11-09  9.23  9.23  9.23   9.23   ----
16   2018-11-07  9.35  9.35  9.35   9.35   ----
17   2018-11-06  9.24  9.24  9.24   9.24   ----
18   2018-11-05  9.18  9.18  9.18   9.18   ----
19   2018-11-02  9.24  9.24  9.24   9.24   ----
20   2018-11-01  9.24  9.24  9.24   9.24   ----
21   2018-10-31  9.18  9.18  9.18   9.18   ----
22   2018-10-30  9.20  9.20  9.20   9.20   ----
23   2018-10-29  9.21  9.21  9.21   9.21   ----
24   2018-10-26  9.19  9.19  9.19   9.19   ----
25   2018-10-25  9.19  9.19  9.19   9.19   ----
26   2018-10-24  9.18  9.18  9.18   9.18   ----
27   2018-10-23  9.19  9.19  9.19   9.19   ----
28   2018-10-22  9.28  9.28  9.28   9.28   ----
29   2018-10-19  9.27  9.27  9.27   9.27   ----
...         ...   ...   ...   ...    ...    ...
983  2015-02-12  9.11  9.11  9.11   9.11   ----
984  2015-02-11  9.07  9.07  9.07   9.07   ----
985  2015-02-10  9.13  9.13  9.13   9.13   ----
986  2015-02-09  9.14  9.14  9.14   9.14   ----
987  2015-02-06  9.27  9.27  9.27   9.27   ----
988  2015-02-05  9.27  9.27  9.27   9.27   ----
989  2015-02-04  9.20  9.20  9.20   9.20   ----
990  2015-02-03  9.15  9.15  9.15   9.15   ----
991  2015-02-02  9.15  9.15  9.15   9.15   ----
992  2015-01-30  9.07  9.07  9.07   9.07   ----
993  2015-01-29  9.11  9.11  9.11   9.11   ----
994  2015-01-28  9.24  9.24  9.24   9.24   ----
995  2015-01-27  9.26  9.26  9.26   9.26   ----
996  2015-01-26  9.27  9.27  9.27   9.27   ----
997  2015-01-23  9.29  9.29  9.29   9.29   ----
998  2015-01-22  9.21  9.21  9.21   9.21   ----
999  2015-01-21  9.24  9.24  9.24   9.24   ----
1000 2015-01-20  9.18  9.18  9.18   9.18   ----
1001 2015-01-19  9.19  9.19  9.19   9.19   ----
1002 2015-01-16  9.24  9.24  9.24   9.24   ----
1003 2015-01-15  9.21  9.21  9.21   9.21   ----
1004 2015-01-14  9.16  9.16  9.16   9.16   ----
1005 2015-01-13  9.18  9.18  9.18   9.18   ----
1006 2015-01-12  9.07  9.07  9.07   9.07   ----
1007 2015-01-09  9.04  9.04  9.04   9.04   ----
1008 2015-01-08  9.01  9.01  9.01   9.01   ----
1009 2015-01-07  8.93  8.93  8.93   8.93   ----
1010 2015-01-06  8.82  8.82  8.82   8.82   ----
1011 2015-01-05  8.88  8.88  8.88   8.88   ----
1012 2015-01-02  8.92  8.92  8.92   8.92   ----

[1013 rows x 6 columns]